In [ ]:
!pip install workalendar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 2.1 MB/s eta 0:00:00
  Created wheel for pymeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=732003 sha256=447f0ffb76ee5f920b7bc4ef48bb633d791454541fa196b23e39caf27e51b084
  Stored in directory: /root/.cache/pip/wheels/d6/67/78/aa2e8d108639dd23a5e9e72a4fc88bb44f5541894382712f48
Successfully built pymeeus


In [ ]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

import pandas as pd

import datetime as dt
from datetime import datetime

from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

#from workalendar.america import Brazil
#calendar = Brazil()

savepath = '/content/drive/MyDrive/DEV/Python/Datasets/TCC/TW-BKP/'

def get_UID():
  curdt = dt.datetime.now().astimezone(dt.timezone(dt.timedelta(hours=-3)))
  return curdt.strftime('%d%m%Y%H%M%S')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAFWmVwEAAAAA2j3NAasSkAcTzqI57kJ4qMd4J7I%3DuPjd93QYDfK4vdKslDBD1L6hPKdvds8Kh568oaqkjHx77gc0Br'

def auth():
    return os.getenv('TOKEN')

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def create_url(keyword, start_date, end_date, max_results = 10):

    search_url = "https://api.twitter.com/2/tweets/search/all"

    # Change to the endpoint you want to collect data from

    # change params based on the endpoint you are using
    #query_params = {'query': keyword,
    #                'start_time': start_date,
    #                'end_time': end_date,
    #                'max_results': max_results,
    #                'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
    #                'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
    #                'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
    #                'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
    #                'next_token': {}}

    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
    #               'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'text,created_at', #'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
    #               'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
    #               'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
    #               'next_token': {}
                    }

    return (search_url, query_params)

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

bearer_token = auth()
headers = create_headers(bearer_token)

# TEST

In [ ]:
#Inputs for the request

keyword = "from:Ambev -is:reply lang:pt"
start_time = "2019-01-01T00:00:00.000Z"
end_time = "2020-01-01T00:00:00.000Z"
user_fields_username = ""
max_results = 100

In [ ]:
url = create_url(keyword, start_time,end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])
#print(json.dumps(json_response, indent=4, sort_keys=True))

Endpoint Response Code: 200


In [ ]:
tw_data = json_response['data']

tw_dates = []
tw_texts = []

print(tw_data[2])

for i in range(len(tw_data)):
  print(json_response['data'][i]['created_at'].split('T')[0]+" | "+json_response['data'][i]['text'])

# CODE

In [ ]:
def get_tweets(date_start, date_end, filename, query='', userlist='none', interval='3M'):
  _tweets = []

  keyword = ''

  if userlist == 'none':
    keyword = "("+query+") lang:pt -is:reply -holidays"
  else:
    if query != '':
      query = "("+query+") "

    for username in userlist:
      keyword += query+"from:"+username+" "
    keyword += "lang:pt -is:reply -holidays"
  max_results = 100

  print(keyword)
  pass
  datelist = pd.date_range(start=date_start, end=date_end, freq=interval).to_list()

  dto = datetime.strptime(date_end, '%Y-%m-%d')
  datelist.append(dto)

  prevdate = date_start
  print(datelist)

  # begin for
  for date_item in datelist:
    curdate = str(date_item.date())

    start_time = prevdate+"T00:00:00.000Z"
    end_time = curdate+"T00:00:00.000Z"

    url = create_url(keyword, start_time,end_time, max_results)
    json_response = connect_to_endpoint(url[0], headers, url[1])
    time.sleep(60)

    print(json_response)

    tw_data = ""
    try:
      tw_data = json_response['data']
      # begin for
      for i in range(len(tw_data)):
        cdate = tw_data[i]['created_at'].split('T')[0]
        ctext = tw_data[i]['text']

        _tweet = {
            'date':cdate,
            'text':ctext
        }
        _tweets.append(_tweet)
        # end for
    except:
      tw_data = json_response['meta']

    prevdate = curdate
  # end for

  df_tw = pd.DataFrame(_tweets)
  df_tw = df_tw.sort_values(by=['date'])

  ffname = "raw_tw_"+filename+"_"+date_start+"_"+date_end+".csv"
  df_tw.to_csv(savepath+ffname, sep=',', index=False)

  return df_tw

In [ ]:
emp = 'petrobras'
#emp = 'valenobrasil'
#emp = 'Bradesco'
#emp = 'Ambev'
#emp = 'itau'

In [ ]:
tw_data = get_tweets('2019-01-01', '2021-01-01','ABEV3', query='ABEV3')

In [ ]:
tw_data = get_tweets('2006-04-01', '2022-01-01','petrobras', userlist=['petrobras'], interval='3M')

In [ ]:
tw_data = get_tweets('2020-01-01', '2021-01-01','petrobras', userlist=['petrobras'])

In [ ]:
stock_list = [['PETR4', 'petrobras OR PETR4'],
              ['VALE3', '"vale do rio docê" OR "VALE3" OR "minério"'],
              ['ABEV3', 'ambev OR ABEV3'],
              ['ITUB4', 'itaú OR ITUB4'],
              ['BBDC4', 'bradesco OR BBDC4']]

journals = ['InvestingBrasil',
            'leiamoneytimes',
            'infomoney',
            'valoreconomico']



In [ ]:
sfstart = '2008-01-01'
sfend = '2022-09-20'

stock_idx = 1

savepath += get_UID()+"/"

if not os.path.exists(savepath):
    os.makedirs(savepath)

for journal in journals:
  tw_data = get_tweets(sfstart, sfend, stock_list[stock_idx][0]+'_'+journal, userlist=[journal], query=stock_list[stock_idx][1])

#InvestingBrasil
#tw_data = get_tweets(sfstart, sfend, stock_list[stock_idx][0]+'_InvestingBrasil', userlist=['InvestingBrasil'], query=stock_list[stock_idx][1])

#leiamoneytimes
#tw_data = get_tweets(sfstart, sfend, stock_list[stock_idx][0]+'_leiamoneytimes', userlist=['leiamoneytimes'], query=stock_list[stock_idx][1])

#infomoney
#tw_data = get_tweets(sfstart, sfend, stock_list[stock_idx][0]+'_infomoney', userlist=['infomoney'], query=stock_list[stock_idx][1])

#valoreconomico
#tw_data = get_tweets(sfstart, sfend, stock_list[stock_idx][0]+'_valoreconomico', userlist=['valoreconomico'], query=stock_list[stock_idx][1])

In [ ]:
tw_data = get_tweets('2008-01-01', '2022-08-31','PETR4', query='PETR4')

In [ ]:
datelist2 = pd.date_range(start='2021-01-01', end='2022-01-01', freq='3M').to_list()

date_str = '2022-01-01'
dto = datetime.strptime(date_str, '%Y-%m-%d')

datelist2.append(dto)

for date_item in datelist2:
  print(date_item.date())

# MERGE DATA

In [ ]:
curstock = 1

merge_files = ['raw_tw_'+stock_list[curstock][0]+'_InvestingBrasil_2008-01-01_2022-09-20.csv',
               'raw_tw_'+stock_list[curstock][0]+'_infomoney_2008-01-01_2022-09-20.csv',
               'raw_tw_'+stock_list[curstock][0]+'_leiamoneytimes_2008-01-01_2022-09-20.csv',
               'raw_tw_'+stock_list[curstock][0]+'_valoreconomico_2008-01-01_2022-09-20.csv']



df_files = []
for item in merge_files:
  df_files.append(pd.read_csv(item))
# -----------

df_merge = pd.concat(df_files, ignore_index=True)
df_merge = df_merge.apply(lambda x: x.astype(str).str.lower()).drop_duplicates(subset=['text'], keep=False)
df_merge['date'] = pd.to_datetime(df_merge['date'])
df_merge = df_merge.sort_values(by='date')

df_merge.to_csv('tw_'+stock_list[curstock][0]+'.csv', sep=',', index=False)



print(df_merge.head())

           date                                               text
4459 2009-04-28  produção de minério de ferro da vale cai 37% h...
4460 2009-05-18  preço de minério sai nas próximas semanas http...
4461 2009-05-26  rio tinto aceita desconto de 33% no minério ht...
4462 2009-06-10  vale acerta preço de minério de ferro com nipp...
4463 2009-07-17  vale fecha desconto no preço do minério para i...


In [ ]:
for item in df_files:
  print(len(item))

print(len(df_merge))

#df_merge = df_merge.drop_duplicates(keep=False)
#df_merge = df_merge.drop_duplicates(subset=["text"], keep=False)

df_merge = df_merge.apply(lambda x: x.astype(str).str.lower()).drop_duplicates(subset=['text'], keep=False)


print(len(df_merge))

1931
4614
2274
3580
12399
12214


In [ ]:
df_files[0].tail()

,date,text
1926,2022-09-12,"#Petrobras reduz preço do gás de cozinha em 4,..."
1927,2022-09-16,CEO da #Petrobras é diagnosticado com câncer -...
1928,2022-09-16,Petrobras capta R$3 bi com emissão de notas co...
1929,2022-09-19,Goldman Sachs: Petrobras com margens consolida...
1930,2022-09-19,"#Petrobras reduz preço do diesel em 5,78% para..."
